In [1]:
import gensim
f = open('C:/Users/adity/Downloads/Yo Yo Honey Singh.txt', 'r')
docs=f.read().split("\n")


In [19]:
import numpy as np
import gensim
import string

from keras.callbacks import LambdaCallback
from keras.layers.recurrent import LSTM
from keras.layers.embeddings import Embedding
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.utils.data_utils import get_file
from keras.layers import Dropout

max_sentence_len = 40
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in docs]
print('Num sentences:', len(sentences))
print(sentences)


Num sentences: 645
[[], [], ['l-ungi'], ['king', 'khan'], ['yo', 'yo', 'honey', 'singh'], [], ['muchho', 'ko', 'thoda', 'round', 'ghumake'], ['anna', 'ke', 'jaise', 'chashma', 'laga', 'ke'], ['coconut', 'me', 'lassi', 'mila', 'ke'], ['aa', 'jao', 'saare', 'mood', 'banaake'], [], ['mucho', 'ko', 'thoda', 'round', 'ghumake'], ['anna', 'ke', 'jaisa', 'chashma', 'lagaake'], ['coconut', 'mai', 'lasi', 'milaake'], ['aajao', 'saare', 'mood', 'banaake'], [], ['all', 'the', 'rajni', 'fans'], ['dont', 'miss', 'the', 'chance'], ['all', 'the', 'rajni', 'fans'], ['dont', 'miss', 'the', 'chance'], ['who', 'is', 'this?'], [], ['lungi', 'dance,', 'lungi', 'dance'], ['lungi', 'dance,', 'lungi', 'dance'], ['lungi', 'dance,', 'lungi', 'dance,'], ['lungi', 'dance,', 'lungi', 'dance', '-', '[2]'], [], ['jado', 'jawaani', 'badaa', 'jor', 'seeve', 'jaad', 'ma'], ['main', 'ta', 'rajni', 'daa', 'fan', 'seeve', 'daad', 'ma'], ['jado', 'jawaani', 'badaa', 'jor', 'seeve', 'jaad', 'ma'], ['main', 'ta', 'rajni', 'd

In [26]:
print('\nTraining word2vec...')
word_model = gensim.models.Word2Vec(sentences, size=100, min_count=1, window=5, iter=100)
pretrained_weights = word_model.wv.syn0
vocab_size, emdedding_size = pretrained_weights.shape
print('Result embedding shape:', pretrained_weights.shape)
print('Checking similar words:')
for word in ['tujhe','meri']:
    most_similar = ', '.join('%s (%.2f)' % (similar, dist) for similar, dist in word_model.most_similar(word)[:8])
    print('  %s -> %s' % (word, most_similar))


def word2idx(word):
    return word_model.wv.vocab[word].index
def idx2word(idx):
    return word_model.wv.index2word[idx]




Training word2vec...
  tujhe -> maanga (0.99), baba (0.99), peer (0.99), bataaya (0.98), pehle (0.98), maine (0.98), manga (0.98), bataya (0.98)
  meri -> khinch, (0.97), kismat (0.97), phooti (0.97), photo (0.97), hogi (0.97), teri (0.96), qayamat (0.95), aankhe (0.94)


C:\Users\adity\AppData\Roaming\Python\Python36\site-packages\ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  


In [4]:

print('\nPreparing the data for LSTM...')
f = open('C:/Users/adity/Downloads/Yo Yo Honey Singh.txt', 'r')
docs=f.read().split("\n")
max_sentence_len = 40
sentences = [[word for word in doc.lower().translate(string.punctuation).split()[:max_sentence_len]] for doc in docs]

train_x = np.zeros([len(sentences), max_sentence_len], dtype=np.int32)
train_y = np.zeros([len(sentences)], dtype=np.int32)
for i, sentence in enumerate(sentences):
    for t, word in enumerate(sentence[:-1]):
        train_x[i, t] = word2idx(word)
        train_y[i] = word2idx(sentence[-1])
print('train_x shape:', train_x.shape)
print('train_y shape:', train_y.shape)


Preparing the data for LSTM...
train_x shape: (645, 40)
train_y shape: (645,)


In [21]:
model = Sequential()
model.add(Embedding(input_dim=vocab_size, output_dim=emdedding_size, 
                    weights=[pretrained_weights]))
model.add(LSTM(units=emdedding_size))
model.add(Dropout(0.2))
model.add(Dense(units=vocab_size))
model.add(Activation('softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy')

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [22]:
def sample(preds, temperature=1.0):
    if temperature <= 0:
        return np.argmax(preds)
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

def generate_next(text, num_generated=10):
    word_idxs = [word2idx(word) for word in text.lower().split()]
    for i in range(num_generated):
        prediction = model.predict(x=np.array(word_idxs))
        idx = sample(prediction[-1], temperature=0.7)
        word_idxs.append(idx)
    return ' '.join(idx2word(idx) for idx in word_idxs)

In [23]:
generate_next('tujhe',10)

'tujhe huyi leke aa shot it, then lagti aajao penthouse ai'

In [24]:
def on_epoch_end(epoch, _):
    print('\nGenerating text after epoch: %d' % epoch)
    texts = ['Teri aankhein']
    for text in texts:
        sample = generate_next(text)
        print('%s... -> %s' % (text, sample))
model.fit(train_x, train_y,
          batch_size=128,
          epochs=40,
          callbacks=[LambdaCallback(on_epoch_end=on_epoch_end)])

Epoch 1/40
645/645 [==============================] - 1s 2ms/step - loss: 6.5239

Generating text after epoch: 0
Teri aankhein... -> teri aankhein nachdi honey jaawani you haskar aankhein all nahi need.. aakhir
Epoch 2/40
645/645 [==============================] - 0s 721us/step - loss: 6.1099

Generating text after epoch: 1
Teri aankhein... -> teri aankhein keemat singh... thi qayamat sai muk bachaunga gaadi na ishare
Epoch 3/40
645/645 [==============================] - 0s 716us/step - loss: 5.6652

Generating text after epoch: 2
Teri aankhein... -> teri aankhein gaadi lagta aakhir thi coconut o.. it.. nache kahaan haa..
Epoch 4/40
645/645 [==============================] - 0s 685us/step - loss: 5.1880

Generating text after epoch: 3
Teri aankhein... -> teri aankhein yeh tennu pe da chhad he.. kar kahaan legi karwa
Epoch 5/40
645/645 [==============================] - 0s 750us/step - loss: 4.8042

Generating text after epoch: 4
Teri aankhein... -> teri aankhein dar jo that, hottie haa

645/645 [==============================] - 0s 625us/step - loss: 4.4935

Generating text after epoch: 39
Teri aankhein... -> teri aankhein hal quicker now passport hottie karna fine.. thoda yacht lo


In [33]:
generate_next('Love')

'love book trust ye ho, sambhalega chal hoon jaan laga ji'